In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

%matplotlib inline

# (A) Nearest Stations

In [3]:
cd geocoding/

/Users/mohiuddi/Desktop/Courses/ADA/solarity/geocoding


In [4]:
import geocoding_functions as geo_fns
import location_functions as loc_fns
import station_info

## User input

In [5]:
address = "Lausanne"

## Parameters

In [6]:
k = 3 ##Number of nearest neighbouring stations

### Get coordinates of the address

In [7]:
coordinates = geo_fns.getCoordinates(address)
if(coordinates is None):
    print("*** Invalid Address ***")
    print("Sample address: Route Cantonale 33, 1025 St. Sulpice")
    exit(0)

In [8]:
coordinates

(46.5196535, 6.6322734)

### Read pickled station info

In [9]:
station_df = pickle.load(open("station_df.p","rb"))

In [25]:
len(station_df)

143

In [10]:
station_df.head()

,name,lat,lng
0,NAP,47.000000,7.933333
1,TAE,47.483333,8.900000
2,MTE,46.150000,7.616667
3,WSLNAF,46.666667,10.233333
4,UEB,47.350000,8.483333


### Get k nearest stations and their weights

In [11]:
neighbourNames = loc_fns.get_k_nearest_neighbours(coordinates, k, station_df)
neighbourWeights = loc_fns.get_weights_for_k_nearest(coordinates, k, station_df)

In [12]:
neighbourNames

45        PUY
17     WSLLAF
105       VEV
Name: name, dtype: object

In [13]:
neighbourWeights

array([ 0.60075651,  0.29506387,  0.10417962])

In [14]:
cd ..

/Users/mohiuddi/Desktop/Courses/ADA/solarity


# (B) Solar potential

The solar potential of each stations has been precomputed. Now, we load it and extract the potential of the nearest stations. We use the extracted solar potentials and weights of the stations to compute the net solar potential of the given address.

In [23]:
meansAllFeatures = pickle.load(open('learning/groupedStationsAllFeatures.p', 'rb'))

In [ ]:
def get_monthly_power_potential(stations, weights, means):
    potential = 0
    for i in range(len(stations)):
        potential = potential + means.power[stations[i]].get_values() * weights[i]
    
    return potential

In [ ]:
monthly_potentialAllFeatures = get_monthly_power_potential(neighbourNames.get_values(), neighbourWeights, meansAllFeatures)

In [ ]:
potentialAllFeatures = np.mean(np.array(monthly_potential)) ## in Wp
potentialAllFeatures

# (C) Installation Costs

## User input

In [ ]:
averageElectricityBill = np.array([80]) ## CHF
roofArea = 30 ## m2

## Parameters

#### Electricity

In [ ]:
elecTarrif = 0.20 ##CHF/Kwh

#### Solar

In [ ]:
capacityPerPanel = 255 ## Wp
costPerPanel = 500 ## CHF
panelArea = 1.62 ## m2

## Average Consumption

In [ ]:
hours = 24
days = 30
milli = 1000

In [ ]:
avgEnergy = averageElectricityBill / elecTarrif
avgEnergy

In [ ]:
avgPower = avgEnergy / (hours*days) ## Kw
avgPower

## Solar Cost and Capacity

In [ ]:
installation = np.array(range(5,101,5)) / 100

In [ ]:
numPanels = np.floor(roofArea/panelArea)

In [ ]:
capacity = installation * numPanels * capacityPerPanel / 1000 ## Kwp
cost = installation * numPanels * costPerPanel ## CHF
solarPower = capacity * potential ## in Kw

In [ ]:
solarPower

### Finding suitable installations

We assume that the startegy of the household is to breakeven. So, the suitbale installation of solar panels will be ones where the solar capacity is less than average power consumption of the household.

In [ ]:
indx = solarPower <= avgPower
installationFinal = installation[indx]
installCapacity = capacity[indx]
installCost = cost[indx]
installPower = solarPower[indx]

In [ ]:
solar_df = pd.DataFrame()
solar_df['capacity'] = installCapacity
solar_df['cost'] = installCost
solar_df['power'] = installPower
solar_df['installation'] = installationFinal
solar_df = solar_df.set_index('installation')
solar_df

# (D) Savings and break-even time

Here, we compute the new electricity bill that he household would incur, after solar installations.

In [ ]:
def get_new_bill(bill, power):
    avgEnergyUsed = bill / elecTarrif
    avgEnergyUsed_daytime = avgEnergyUsed / 2
    nightUsage = avgEnergyUsed / 2

    avgEnergyProduced = power * hours * days
    
    newBill = np.empty(len(power))
    newUsageFromUtility = np.empty(len(power))
    for i in range(len(power)):
        dayUsage = max(0, avgEnergyUsed_daytime - avgEnergyProduced[i])
        newUsageFromUtility[i] = dayUsage + nightUsage
        newBill[i] = newUsageFromUtility[i] * elecTarrif

    return newBill, newUsageFromUtility

In [ ]:
newBill, newUsage = get_new_bill(averageElectricityBill, installPower)

Here, savings are computed. savings_for_an_installation = new_bill - old_bill - cost_of_installation.
We compute the monthly savings for a period of 25 years.

In [ ]:
totalYears = 25
monthsInAYear = 12
def get_cummulative_savings(oldBill, newBill, cost):
    months = range(1, totalYears*monthsInAYear + 1)
    savings = np.empty((len(newBill), len(months)))
    for i in range(len(newBill)):
        eachMonth = oldBill - newBill[i]
        savings[i,:] = eachMonth*months - cost[i]
    return savings

In [ ]:
savings = get_cummulative_savings(averageElectricityBill, newBill, installCost)

In [ ]:
finalSavings = savings[:,-1]

From the monthly savings over a period of 25 years, we compute the break-even time.

In [ ]:
def get_break_even_time(savings):
    nInstalls = savings.shape[0]
    breakEven = np.empty(nInstalls)
    for i in range(nInstalls):
        breakEven[i] = np.argmax(savings[i,:] > 0)

    return breakEven

In [ ]:
breakEven = get_break_even_time(savings)/12

### Prettify Result

In [ ]:
newBill

In [ ]:
final_df = solar_df
final_df['breakEvenYears'] = breakEven
final_df['savings_after_25yrs'] = finalSavings
final_df['annual_savings'] = finalSavings/totalYears
final_df

### Plot savings

In [ ]:
for i in range(savings.shape[0]):
    plt.plot(savings[i,:])